In [ ]:
import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [17]:
##load haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('C:\\Users\\Dell\\Project-2.2_Digital-Assistant\\src\\main\\java\\FacialRecognision\\haarcascade_frontalface_default.xml')
if(face_cascade.empty() or face_cascade is None):
    print("cascade not found")
    exit(1)

In [10]:



##change for correct path
path="C:\\Users\\Dell\\Project-2.2_Digital-Assistant\\src\\main\\resources\\Facedataset\\Humans"
images=[]

for file in os.listdir(path):
    img=cv2.imread(os.path.join(path,file))
    images.append(img)


In [21]:
##find 200 faces the classifier detects

faces=[]
indexes=[]
index=0
for img in images:
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    face=face_cascade.detectMultiScale(gray,1.1,4)
    if(len(face)>0):
        faces.append(img)
        indexes.append(index)
    if len(faces)>=200:
        break
    if len(faces)%10==0:
        print(len(faces))
    index+=1


In [25]:
##define noise methods all methods assume grayscale images

def scale_gaussian_noise(image, scaling_factor):
   
    original_sigma = 1.0  # Original standard deviation
    scaled_sigma = original_sigma * (scaling_factor / 100)
    
    noise = np.random.normal(0, original_sigma, image.shape)
    
    scaled_noise = noise * scaled_sigma
    
    noisy_image = image + scaled_noise
    

    noisy_image = np.clip(noisy_image, 0, 255)

    noisy_image = noisy_image.astype(np.uint8)
    
    return noisy_image


def salt_pepper_noise(image,percentage):
    newImage=np.copy(image)
    row,col = image.shape
    amountofpixels = percentage*row*col
    for i in range(int(amountofpixels)):
        x=np.random.randint(0,row-1)
        y=np.random.randint(0,col-1)
        random = np.random.randint(0,2)
        if random == 0:
            newImage[x][y] = 0
        else:
            newImage[x][y] = 255
    return newImage

def scale_poisson_noise(image, scaling_factor):

    original_lambda = 10.0  
    scaled_lambda = original_lambda * (scaling_factor / 100)
    
    noise = np.random.poisson(scaled_lambda, image.shape).astype(np.float32)
    
    # Add the noise to the image
    noisy_image = image + noise
    
    # Clip the pixel values to the valid range of [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255)
    
    # Convert the pixel values back to integers
    noisy_image = noisy_image.astype(np.uint8)
    
    return noisy_image


In [27]:
##now we have 200 faces we can add noise to them and test the classifier
import time
start=time.time()
accuracyGaussian=[]
accuracySaltPepper=[]
accuracyPoisson=[]
for i in range(0,100,10):
    print("current noise percentage: "+str(i))
    accuracyGaussiantemp=0
    accuracySaltPeppertemp=0
    accuracyPoissontemp=0
    for img in faces:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gaussian=scale_gaussian_noise(img,i)
        saltpepper=salt_pepper_noise(img,i/100)
        poisson=scale_poisson_noise(img,i)
        detected=face_cascade.detectMultiScale(gaussian,1.1,4)
        if(len(detected)>0):
            accuracyGaussiantemp+=1
        detected=face_cascade.detectMultiScale(saltpepper,1.1,4)
        if(len(detected)>0):
            accuracySaltPeppertemp+=1
        detected=face_cascade.detectMultiScale(poisson,1.1,4)
        if(len(detected)>0):
            accuracyPoissontemp+=1
    accuracyGaussian.append([i,accuracyGaussiantemp/len(faces)])
    accuracySaltPepper.append([i,accuracySaltPeppertemp/len(faces)])
    accuracyPoisson.append([i,accuracyPoissontemp/len(faces)])
    if i==0:
        print("time to calculate 0% noise: "+str(time.time()-start)+" expected time: "+str((time.time()-start)*10)+" seconds")
##plot the results
plt.plot(*zip(*accuracyGaussian),label="Gaussian")
plt.plot(*zip(*accuracySaltPepper),label="Salt and Pepper")
plt.plot(*zip(*accuracyPoisson),label="Poisson")
plt.legend()
plt.xlabel("Noise Percentage")
plt.ylabel("Accuracy")
plt.title("Accuracy of Face Detection with Noise")
plt.show()






current noise percentage: 0
time to calculate 0% noise: 552.3452515602112 expected time: 5523.452515602112 seconds
current noise percentage: 10
current noise percentage: 20
current noise percentage: 30


KeyboardInterrupt: 